In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from trl import SFTTrainer
from transformers import TrainingArguments
from tuning.data.train_dataset import get_train_dataset
from tuning.training.config_training import ModelLoadConfig, LoraConfig, SFTRunConfig, PTRunConfig, DPOTrainingConfig, TrainingArgumentsConfig, PassAtKConfig, sft_batch_size, effective_batch_size
from tuning.training.perplexity_callback import PerplexityStoppingCallback
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.utils.utils import apply_chat_template, chat_template_func
import json
import sys
from datasets import load_from_disk
from typing import List, Optional, Union
from pathlib import Path
from tuning.config import DATASETS_DIR, HF_MODEL_MAP
import os
from tuning.training.config_training import DatasetConfig, SFTRunConfig
from tuning.config import MODELS_DIR
from tuning.training.sft_training import train_model_sft
from tuning.training.dpo_training import train_model_dpo
import subprocess
import importlib
import torch
import json
import wandb


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-07 16:09:12 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annota

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [2]:

  import importlib
  from tuning.training import passk_callback, sft_training
  importlib.reload(passk_callback)


<module 'tuning.training.passk_callback' from '/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/training/passk_callback.py'>

In [3]:
MODEL = "llama3-1B"
total_train_size = 4096  # 29980
perplexity_thresholds = [7.0,6.0, 5.75, 5.5, 5.25, 5.0, 4.75, 4.5, 4.25,4.0, 3.9, 3.8, 3.7, 3.6,3.55,3.5,3.45,3.4,3.35,3.3, 3.25, 3.2, 3.15, 3.1]
perplexity_thresholds = [6.0, 5.0]

In [ ]:
dataset_config = DatasetConfig(
    dataset = "tuluif",
    dataset_type = "sft",
    train_size = total_train_size, # 29980
)

run_config = SFTRunConfig(
    dataset_config = dataset_config,
    model_name_hf = HF_MODEL_MAP[MODEL],  # Use HuggingFace model name, not local path
    model_name = MODEL,  # Base model name for output directory construction
    do_training=True,
    do_inference=False,
    do_evaluation=False,
)
passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
    target_pass_at_k=[0.1, 0.15, 0.2,0.25,0.3],
    k_values=[1],
    n_samples=1,
    num_prompts=541,
    temperature=0.7,
    strict=False,
    enabled=True,
)

lora_config = LoraConfig()
model_load_config = ModelLoadConfig()
model_load_config.max_seq_length = 4096
training_args = TrainingArgumentsConfig()
training_args.eval_steps = 1

In [5]:
try:
    wandb.finish()
except Exception:
    pass
run = wandb.init(
    name=run_config.run_name, 
    project="tuning", 
    # reinit=True,
    # Optional: Pass config here so it's logged even if training crashes early
    config=run_config.__dict__ if hasattr(run_config, "__dict__") else {} 
)

with run:
    model, tokenizer, trainer, callbacks = train_model_sft(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        passk_config = passk_config
    )   

wandb: Currently logged in as: shougan (shougan-university-of-waterloo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Getting train dataset for run config: llama3-1B_sft-tuluif-4096
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-4096
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-4096
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 4096
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_se11a0hu4d0lnuekd4d6hmmb', 'prompt': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have received significant citations. Provide a brief summary of each paper, not exceeding 50 words per summary.', 'messages': [{'content': 'You are a helpful assi

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 4096
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 200
    })
})
{'id': 'personas_IF_se11a0hu4d0lnuekd4d6hmmb', 'prompt': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have received significant citations. Provide a brief summary of each paper, not exceeding 50 words per summary.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'Identify and list three groundbreaking research papers in the field of artificial intelligence from the past five years. The papers should be recognized for their innovative contributions and have rece

Unsloth 2025.10.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


...

Unsloth: Copying 1 files from cache to `/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B_sft-tuluif-4096`: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


Successfully copied all 1 files from cache to `/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B_sft-tuluif-4096`


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:06<00:00,  6.84s/it]


Unsloth: Merge process complete. Saved to `/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B_sft-tuluif-4096`


eval/loss,█████▇▆▆▅▅▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/pass_at_1,▄▆▅▃▆█▂▆▇▆▃▅▅▅▃▂▃▅▆▁▅▄▂▃██▃▅▃▆▇▆▆▂▆▄▆▆▆▄
eval/runtime,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁█████▇███████████████████▆███▃█████████
eval/steps_per_second,▁███▇█▇██████████▆████████▆█████▃██▇████
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇████
train/global_step,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▆▇▇▇█▇▄▅▅▄▃▃▃▄▃▂▂▂▃▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▃▄▅▆█████████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,███▇█▆▅▅▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▁▁▁▂▂▂
eval/loss,1.47445


In [6]:
# ppl_callback = callbacks[-1]
# metadata_file = ppl_callback.metadata_path
metadata_file = "/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models_metadata/llama3-8B_passatk-0201_2257.json"
checkpoints = []
with open(metadata_file, "r") as f:
    for line in f:
        checkpoints.append(json.loads(line))
print(checkpoints)


[{'threshold_type': 'pass_at_1', 'threshold_value': 0.15, 'global_step': 1, 'checkpoint_path': '/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B_pass@1-0.15_sft-64', 'data_points_seen': 64, 'k_value': 1, 'n_samples': 1, 'strict': False}]


In [7]:
!nvidia-smi
import gc
# del model, tokenizer, trainer, callbacks # this deletes the references to such objects
gc.collect() # then we force the GC
torch.cuda.empty_cache() # and we release the GPU CUDA cache
!nvidia-smi
# we could also use reset_cuda_context() for a clean state

Sat Feb  7 15:56:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:5D:00.0 Off |                    0 |
| N/A   38C    P0            120W /  700W |    4611MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [13]:
for checkpoint in checkpoints:
    try:
        wandb.finish()
    except Exception:
        pass
    
    model_name = Path(checkpoint["checkpoint_path"]).name
    data = total_train_size - checkpoint["data_points_seen"] 
    model_load_config = ModelLoadConfig()
    training_args = DPOTrainingConfig()
    # training_args.eval_strategy = "epoch"  # Evaluates at end of each epoch
    # training_args.load_best_model_at_end = False  # Keep latest model
    # training_args.report_to = []  # Disable all logging integrations including W&B
    training_args.per_device_train_batch_size = 8
    training_args.gradient_accumulation_steps = 16
    training_args.eval_steps = 4
    dataset_config = DatasetConfig(
        dataset = "tuluif",
        dataset_type = "pt",
        train_size = data,
    )
    sft_run_config = SFTRunConfig(
        dataset_config = DatasetConfig(
            dataset = "tuluif",
            dataset_type = "sft",
            train_size = checkpoint["data_points_seen"],
            dynamic_path = model_name
        ),
        model_name = MODEL,
        model_name_hf = HF_MODEL_MAP[MODEL], 
        task_name = "ifeval"
    )
    run_config = PTRunConfig(
        dataset_config = dataset_config,
        model_name_hf = HF_MODEL_MAP[MODEL],  
        model_name = MODEL,  
        sft_run_config = sft_run_config,
        task_name = "ifeval",
        pft_method = "dpo",
        do_training = True
    )
    passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
        target_pass_at_k=[1.2],
        k_values=[1],
        n_samples=1,
        num_prompts=541,
        temperature=0.7,
        strict=False,
        enabled=True,
    )
    model, tokenizer, trainer = train_model_dpo(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        passk_config = passk_config,
        # perplexity_thresholds= [0.1] # dummy value to periodically check perplexities too
    )
    del model, tokenizer, trainer
    gc.collect()
    torch.cuda.empty_cache()




Per device train batch size: 2
Getting train dataset for run config: llama3-1B_llama3-1B_pass@1-0.15_sft-64_pt-tuluif-4032
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-4032
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-4032
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 4032
    })
    test: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_h9fygv5fcwjvn671fe0v5wqv', 'prompt': "What is the best way to overcome writer's block while working late at night on a novel? Choose from the following: ('take a short walk', 'have a cup of coffee', 'meditate 

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,032 | Num Epochs = 2 | Total steps = 64
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 16 x 1) = 128
 "-____-"     Trainable parameters = 22,544,384 of 1,258,358,784 (1.79% trained)


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
!nvidia-smi